In [99]:
import numpy as np
import pandas as pd
import river

from river import compose
from river import preprocessing
from river import linear_model
from river import metrics
from river import evaluate
from river import neighbors
from sklearn.ensemble import RandomForestClassifier

from kafka import KafkaConsumer
from river import multiclass
import skmultiflow

In [100]:
# write the print content to file 
f1 = open("./2-2-6 predictive scores for an adaptive model with initial data.log",'w+')
f2 = open("./2-2-6 predictive scores for an adaptive mode with kafka data.log",'w+')

In [101]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^ get data from sklearn ^^^^^^^^^^^^^^^^^^^^^^^^^^^
# author:           Kun Yan
# student number:   300259303
# data:             2021-10-30
# Python version:   3.9.7
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

# read from sklearn


data_aftcat = pd.read_csv(r"./2-2-1 5388 data_aftcat.csv")
data_y = pd.read_csv(r"./2-2-1 5388 data_aftcat class.csv")
data = pd.read_csv(r"./2-2-1 5388 data_aftcat_full.csv") # class, source(OneHotEncoder)

In [102]:
d_col = data.columns
d_aftcat_col = data_aftcat.columns
y_col = data_y.columns
# change from pd.DataFrame to np.array
data = np.array(data)
data_aftcat = np.array(data_aftcat)
data_y = np.array(data_y)


# train an adaptive model with initial data

In [103]:
# ^^^^^^^^^^^^^^^^^^^^^ train a model using river ^^^^^^^^^^^^^^^^^
# author:           Kun Yan
# student number:   300259303
# data:             2021-10-30
# Python version:   3.9.7
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# train an adaptive model and prediction
mm = river.preprocessing.StandardScaler()
#selector = river.feature_selection.VarianceThreshold()
#model_adaptive = tree.HoeffdingAdaptiveTreeClassifier(grace_period=100,split_confidence=1e-5,leaf_prediction='nb',nb_threshold=10,seed=0)
model_adaptive = neighbors.KNNClassifier() # window_size (int) – defaults to 1000
#model = compose.Pipeline(mm,selector,model_adaptive)
#model = compose.Pipeline(mm,model_adaptive)
#model = river.imblearn.RandomOverSampler((mm | selector| model_adaptive),desired_dist={False: 0.5, True: 0.5})
ovr = multiclass.OneVsRestClassifier(model_adaptive)
model = mm | ovr

In [142]:
metric_acc = metrics.Accuracy()
metric_classr = metrics.ClassificationReport()

In [107]:
print("= = = = = = prediction score for an active model with initial data = = = = = ")
print("= = = = = = prediction score for an active model with initial data = = = = = ",file=f1)

= = = = = = prediction score for an active model with initial data = = = = = 


In [143]:
i = 0
for xi, yi in zip(data_aftcat,data_y):
    xi = dict(zip(d_aftcat_col, xi))   # transfor array to dict
    yi = yi[0]
    yi_pred = model.predict_one(xi)
    metric_acc = metric_acc.update(yi,yi_pred)
    #metric_classr = metric_classr.update(yi,yi_pred)
    model = model.learn_one(xi, yi)
    i=i+1

print("metric_acc:",metric_acc)
#print("metric_classr:",metric_classr)
print("i:",i)

print("metric_acc:",metric_acc,file=f1)
#print("metric_classr",metric_classr,file=f1)
print("i:",i,file=f1)

metric_acc: Accuracy: 97.32%
i: 25284


In [144]:
i = 0
for xi, yi in zip(data_aftcat,data_y):
    xi = dict(zip(d_aftcat_col, xi))   # transfor array to dict
    yi = int(yi[0])
    #print("yi:",yi)
    yi_pred = int(model.predict_one(xi))
    #print("yi_pred",yi_pred)
    #metric_acc = metric_acc.update(yi,yi_pred)
    metric_classr = metric_classr.update(yi,yi_pred)
    model = model.learn_one(xi, yi)
    #break
    i=i+1

In [145]:
metric_classr

           Precision   Recall   F1      Support  
                                                 
       0       0.998    0.996   0.997     21962  
       1       0.000    0.000   0.000        18  
       2       0.875    0.987   0.928       377  
       3       0.746    0.645   0.692         2  
       4       0.291    0.300   0.296       213  
       5       0.572    0.700   0.630       446  
       6       0.188    0.031   0.054        96  
       7       0.657    0.512   0.575        86  
       8       0.864    0.376   0.524       101  
       9       0.939    0.993   0.965      1774  
      10       0.000    0.000   0.000        11  
                                                 
   Macro       0.557    0.504   0.515            
   Micro       0.973    0.973   0.973            
Weighted       0.971    0.973   0.971            

                 97.3% accuracy                  

In [148]:
i = 0
for xi, yi in zip(data_aftcat,data_y):
    xi = dict(zip(d_aftcat_col, xi))   # transfor array to dict
    yi = yi[0]
    #print("yi:",yi)
    yi_pred = model.predict_one(xi)
    
    if yi != None:
        if yi_pred != None:
            #print("yi_pred",yi_pred)
            #metric_acc = metric_acc.update(yi,yi_pred)
            metric_classr = metric_classr.update(yi,yi_pred)
            model = model.learn_one(xi, yi)
    i=i+1
    

In [150]:
print("metric_classr:",metric_classr)
print("i:",i)
print("metric_classr",metric_classr,file=f1)
print("i:",i,file=f1)

metric_classr:            Precision   Recall   F1      Support  
                                                 
       0       0.998    0.996   0.997     43924  
       1       0.000    0.000   0.000        36  
       2       0.874    0.987   0.927       754  
       3       0.744    0.645   0.691         4  
       4       0.294    0.303   0.298       426  
       5       0.574    0.702   0.631       892  
       6       0.188    0.031   0.054       192  
       7       0.654    0.506   0.570       172  
       8       0.862    0.371   0.519       202  
       9       0.939    0.993   0.965      3548  
      10       0.000    0.000   0.000        22  
                                                 
   Macro       0.557    0.503   0.514            
   Micro       0.973    0.973   0.973            
Weighted       0.971    0.973   0.971            

                 97.3% accuracy                  
i: 25284


# train an adaptive model with kafka data

In [151]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^ read kafka data and detect concept drift^^^^^^^^^^
# author:           Kun Yan
# student number:   300259303
# data:             2021-10-30
# Python version:   3.9.7
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# instantiate the KafkaConsumer Class using the arguments mentioned.
# do not change any arguments other than the first positional argument.
consumer = KafkaConsumer(
    'task2',   # change this to "task2" for the IOT Botnet Detection  ---- important ----
    bootstrap_servers="34.130.121.39:9092",
    sasl_plain_username="student",
    sasl_plain_password="uottawa",
    security_protocol="SASL_PLAINTEXT",
    sasl_mechanism="PLAIN",
    auto_offset_reset='earliest',
    enable_auto_commit=False
)

In [152]:
metric_acc_kafka = metrics.Accuracy
metric_cr_kafka= metrics.ClassificationReport()

In [153]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^ function: map Class to 0 - 10^^^^^^^^^^^^^^^^^^^^
# author:           Kun Yan
# student number:   300259303
# data:             2021-10-30
# Python version:   3.9.7
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
def Class_map(d_c):
    if (d_c == 'BENIGN'):        
        y = 0
    else:
        if(d_c == 'mirai_udp_attack'):
            y = 1
        else:
            if(d_c == 'mirai_ack_attack'):
                y = 2
            else:
                if(d_c == 'gafgyt_scan_attack'):
                    y = 3
                else:
                    if(d_c == 'mirai_scan_attack'):
                        y = 4
                    else:
                        if(d_c == 'gafgyt_tcp_attack'):
                            y = 5
                        else:
                            if(d_c == 'gafgyt_udp_attack'):
                                y = 6
                            else:
                                if(d_c == 'gafgyt_junk_attack'):
                                    y = 7
                                else:
                                    if(d_c == 'gafgyt_combo_attack'):
                                        y = 8
                                    else:
                                        if(d_c == 'mirai_syn_attack'):
                                            y = 9
                                        else:
                                            if(d_c == 'mirai_udpplain_attack'):
                                                y = 10
    return y


In [154]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^ function: map Source to OneHotEncoder^^^^^^^^^^^^^^^^^^^
# author:           Kun Yan
# student number:   300259303
# data:             2021-10-30
# Python version:   3.9.7
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
def source_map(d_s):
    sr_dict = {'x0_0': 0.0, 'x0_1': 0.0, 'x0_2': 0.0, 'x0_3': 0.0, 'x0_4': 0.0, 'x0_5': 0.0, 'x0_6': 0.0, 'x0_7': 0.0, 'x0_8': 0.0}
    if(d_s == 'Provision PT 838 Security Camera'):
        sr_dict['x0_0'] = 1.0
    else:
        if(d_s == 'Provision PT 737E Security Camera'):
            sr_dict['x0_1'] = 1.0
        else:
            if(d_s == 'Samsung SNH 1011 N Webcam'):
                sr_dict['x0_2'] = 1.0
            else:
                if(d_s == 'SimpleHome XCS7 1002 WHT Security Camera'):
                    sr_dict['x0_3'] = 1.0
                else:
                    if(d_s == 'Philips B120N10 Baby Monitor'):
                        sr_dict['x0_4'] = 1.0
                    else:
                        if(d_s == 'SimpleHome XCS7 1003 WHT Security Camera'):
                            sr_dict['x0_5'] = 1.0
                        else:
                            if(d_s == 'Ennio Doorbell'):
                                sr_dict['x0_6'] = 1.0
                            else:
                                if(d_s == 'Danmini Doorbell'):
                                    sr_dict['x0_7'] = 1.0
                                else:
                                    if(d_s == 'Ecobee Thermostat'):
                                        sr_dict['x0_8'] = 1.0
    return sr_dict

In [163]:
# Data Stream flowing in. - - - Dict()
i = 0
# concept drift detection
kswin = river.drift.KSWIN(window_size = 1000)
drifts_ksw = []
nan=0
inf=0
for message in consumer:
    #print(message)
    tmp_data=eval(message.value.decode(errors='ignore'))   #dict()
    dict_y = Class_map(tmp_data['Class'])
    dict_source = source_map(tmp_data['Source'])
    del tmp_data['Class']
    del tmp_data['Source']
    dict_data = dict(tmp_data,**dict_source)
    #print("items:",tmp_data.items())
    #print("values:",tmp_data.values())

    # prediction and train an adaptive model
    yi_pred = model.predict_one(dict_data)  # make a prediction
    if dict_y != None:
        if yi_pred != None:
            #metric_acc_kafka = metric_acc_kafka.update(dict_y,yi_pred)
            metric_cr_kafka = metric_cr_kafka.update(dict_y,yi_pred)
            model = model.learn_one(dict_data,dict_y) # make the model learn

    # concept drift detection
    in_drift, in_warning = kswin.update(message.value)
    if in_drift:
        print(f'Drift detected at index {i}')
        print(f'Drift detected at index {i}',file=f2)
        drifts_ksw.append(i)
        kswin.reset()   # As a best practice, we reset the detector   

    if i == 100000:
       break       
    i += 1


Drift detected at index 1007
Drift detected at index 2281
Drift detected at index 3288
Drift detected at index 4371
Drift detected at index 6199
Drift detected at index 7869
Drift detected at index 9518
Drift detected at index 11191
Drift detected at index 12766
Drift detected at index 14737
Drift detected at index 15877
Drift detected at index 17259
Drift detected at index 18755
Drift detected at index 20205
Drift detected at index 21573
Drift detected at index 25158
Drift detected at index 26686
Drift detected at index 27800
Drift detected at index 29040
Drift detected at index 30057
Drift detected at index 31078
Drift detected at index 32937
Drift detected at index 33963
Drift detected at index 35248
Drift detected at index 36316
Drift detected at index 37895
Drift detected at index 39367
Drift detected at index 40845
Drift detected at index 41994
Drift detected at index 43111
Drift detected at index 44114
Drift detected at index 45742
Drift detected at index 46763
Drift detected at

In [164]:
#print("metric_acc_kafka \n",metric_acc_kafka)
print("metric_cr_kafka \n",metric_cr_kafka)
print("i:",i)

#print("metric_cr_kafka \n",metric_cr_kafka,file=f2)
print("metric_acc_kafka \n",metric_acc_kafka,file=f2)
print("i:",i,file=f2)

metric_cr_kafka 
            Precision   Recall   F1      Support  
                                                 
       0       0.998    0.996   0.997     86826  
       1       0.945    0.990   0.967      6956  
       2       0.067    0.006   0.011        32  
       3       0.744    0.639   0.688       793  
       4       0.597    0.534   0.563       399  
       5       0.302    0.308   0.305       883  
       6       0.564    0.692   0.621       168  
       7       0.874    0.981   0.925      1589  
       8       0.200    0.020   0.037        49  
       9       0.732    0.363   0.485       452  
      10       1.000    0.036   0.069        56  
                                                 
   Macro       0.638    0.506   0.515            
   Micro       0.973    0.973   0.973            
Weighted       0.971    0.973   0.971            

                 97.3% accuracy                  
i: 100000


# Method 2: train an adaptive model using skmultiflow

In [168]:
# ^^^^^^^^^^^^^^^^^^^^^ train a model using skmultiflow ^^^^^^^^^^^^^^^^^
# author:           Kun Yan
# student number:   300259303
# data:             2021-10-30
# Python version:   3.9.7
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


from skmultiflow.lazy import KNNADWINClassifier
import skmultiflow
f3 = open("./2-1-6 predictive scores for an adaptive model using skmultiflow.log",'w+')

data = pd.read_csv(r"./2-2-1 5388 data_aftcat.csv")
num = len(data.index)

# Setup the stream
stream = skmultiflow.data.DataStream(data,allow_nan=True)
model_knnadw = KNNADWINClassifier(n_neighbors=8, max_window_size=1000, leaf_size=40)

# Keep track of sample count and correct prediction count
n_samples = 0
corrects = 0
while n_samples < num:
    X, y = stream.next_sample()
    #print("y:",y)
    pred = model_knnadw.predict(X)
    #print("pred:",pred)
    if y[0] == pred[0]:
        corrects += 1
    model_knnadw = model_knnadw.partial_fit(X, y)
    n_samples += 1

In [169]:
print('KNNClassifier usage example')
print(str(n_samples) + ' samples analyzed.')
print("KNNADWINClassifier's performance: " + str(corrects/n_samples))

print('KNNClassifier usage example',file=f3)
print(str(n_samples) + ' samples analyzed.',file=f3)
print("KNNADWINClassifier's performance: " + str(corrects/n_samples),file=f3)

KNNClassifier usage example
25284 samples analyzed.
KNNADWINClassifier's performance: 0.9703369719981015
